In [ ]:
from AHI_AC import *

# Set spatial resolution
# 0.01deg ==> 0.01, 0.02deg ==> 0.02
res = 0.01

# Set latitude upper and bottom bounds
lat_up, lat_bottom = 60, -60

# Set longitude left and right bounds
lon_left, lon_right = 85, 205

# Set save folder
target = '/data01/GEO/SRDATA/'

# See main.py
row,col,row_AHI,col_AHI,AHI_lat,AHI_lon,AHI_VZA, AHI_VAA,AHI_AL,Water_idx,Landmask = main_read(res,lat_up, lat_bottom,lon_left, lon_right)

# Read LUT and interpolate with LUT
# Processing band can be set by yourself!
interpolation = LUT_interpolator(1)
fn1_1, fn2_1, fn3_1 = interpolation._interpolate()
interpolation = LUT_interpolator(2)
fn1_2, fn2_2, fn3_2 = interpolation._interpolate()
interpolation = LUT_interpolator(3)
fn1_3, fn2_3, fn3_3 = interpolation._interpolate()
interpolation = LUT_interpolator(4)
fn1_4, fn2_4, fn3_4 = interpolation._interpolate()

# Set necessary parameters
band_type = {'1': 'vis', '2': 'vis', '3': 'ext', '4': 'vis'}
band_num = {'1': '01', '2': '02', '3': '01', '4': '03'}
calculate_6s = {'1': calculate_6s_band1, '2': calculate_6s_band2, '3': calculate_6s_band3, '4': calculate_6s_band4}
fn1 = {'1': fn1_1, '2': fn1_2, '3': fn1_3, '4': fn1_4}
fn2 = {'1': fn2_1, '2': fn2_2, '3': fn2_3, '4': fn2_4}
fn3 = {'1': fn3_1, '2': fn3_2, '3': fn3_3, '4': fn3_4}


In [ ]:
def SR_Process(band):
    start_time = T.time()

    # Download AHI
    print(f"Start download AHI for Band{band}")
    AHI_data = H8_Process('liwei', 'liwei00', band_type[band], band_num[band], date, folder_original)
    if AHI_data == 'No data':
        print(f"No AHI data for Band{band}, pass")
        return None
    E_Read_AHI_time = T.time()
    print(f"Finish download AHI for Band{band}")

    # SR calculate
    print(f'Start Calculate SR for Band{band}')
    S_SR_time = T.time()
    RESULT = Parallel(n_jobs=njob)(delayed(calculate_6s[band])(fn1[band], fn2[band], fn3[band], Landmask, Aerosol_type, *params[band], AHI_data, i) for i in range(row_AHI))
    SR = np.full((row_AHI,col_AHI), np.nan)
    for i in range(row_AHI):
        SR[i][Water_idx[i]] = RESULT[i]
    E_SR_time = T.time()
    print(f'Calculate SR for Band{band} finished')

    # Save file and remove download input data
    print(f'Start Save SR file for Band{band}')
    S_SAVE_time = T.time()
    SR = np.array(SR).reshape(row_AHI,col_AHI)
    
    SR = np.around(SR, 4) * 10000
    
    SR = np.clip(SR, -32767, 32767)
    SR = np.nan_to_num(SR, nan=-32768)
    SR = np.where(np.isnan(SR), -32768, SR)

    SR_file = open(folder_AC + f'H08_{YYYY}{MM}{DD}_{HH}{MIN}_B0{band}_SR.dat', 'wb')
    SR.astype('int16').tofile(SR_file)
    SR_file.close()

    print(f'Save Band{band} SR finished')
    if band == '4':
        remove_original_file(folder_original)
        print("delete file finish")
    E_SAVE_time = T.time()
    end_time = T.time()
    TIME = end_time - start_time
    print('Band{} cost time: {:} secs, {:.1f} mins\n'.format(band, TIME, TIME / 60))

In [ ]:
njob = 20
date_start = '2018-12-21 00:00'
date_end = '2018-12-21 02:00'
date_t = dt.timedelta(minutes=10)
date_s = dt.datetime.strptime(date_start, "%Y-%m-%d %H:%M")
date_e = dt.datetime.strptime(date_end, "%Y-%m-%d %H:%M")
date_time_now = date_s
# print("Start {} ".format(date_start[:-6]))
YYYY,MM,DD,HH,MIN,date= Time_split(date_time_now)
while date_time_now <= date_e:

    date_dl_str = date_time_now.strftime("%Y-%m-%d %H:%M" )
    YYYY,MM,DD,HH,MIN,date= Time_split(date_time_now)   
    hour = int(date[8:10])
    # print(date_time_now)
    if hour >=2 and hour < 10:
        S_time = T.time()
        print("Start processing {}".format(date))
    
        # make dir
        folder_original = f"{target}{date}_original/"
        folder_AC = f"{target}{YYYY}/{MM}/{DD}/"
        mkdir(folder_original)
        mkdir(folder_AC)

        # Solar angle
        print('Start read solar angle')
        S_ANGLE_time = T.time()
        AHI_SZA,AHI_SAA = AHI_angle(date,row_AHI).read_solar_angle()
        AHI_SZA,AHI_SAA = AHI_SZA[row:row+row_AHI,col:col+col_AHI],AHI_SAA[row:row+row_AHI,col:col+col_AHI]
        RAA = abs(AHI_SAA - AHI_VAA)
        RAA[RAA>180]=360-RAA[RAA>180]
        print('Finish read solar angle')
        E_ANGLE_time = T.time()
        print('Read angle cost : {}s'.format(round((E_ANGLE_time-S_ANGLE_time),4)))
        
        # Atmosphereic data
        print('Start read CAMS')
        S_ATMOS_time = T.time()
        CAMS_data_obj = CAMS_data(YYYY, MM, DD, HH, MIN, AHI_lat, AHI_lon, row_AHI)
        Aerosol_type, OZ, WV, AOT550 = read_cams_data(CAMS_data_obj)
        print('Finish read CAMS')
        E_ATMOS_time = T.time()
        print('Read Atmospheric data cost : {}s'.format(round((E_ATMOS_time-S_ATMOS_time),4)))
        
        params = {
                    '1': [OZ, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL],
                    '2': [OZ, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL],
                    '3': [WV, OZ, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL],
                    '4': [WV, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL]
                }
        
        for band in ['1', '2', '3', '4']:
            SR_Process(band)
        E_time = T.time()
        print('Total time cost : {}s'.format(round((E_time-S_time),4)))
    else:
        print(f"{date_time_now} not in time range, pass")
    date_time_now = date_time_now + date_t